In [1]:
import pandas as pd
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import warnings
warnings.filterwarnings("ignore")
import bs4
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import AIMessage,HumanMessage

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
llm = ChatGroq(temperature=0.3, groq_api_key="", model_name="llama-3.3-70b-versatile")

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"use_auth_token": ""})

In [4]:
loader= WebBaseLoader(web_path="https://langchain-ai.github.io/langgraph/tutorials/introduction/#part-3-adding-memory-to-the-chatbot",default_parser="html.parser")

In [5]:
doc= loader.load()

In [6]:
doc

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/introduction/#part-3-adding-memory-to-the-chatbot', 'title': 'Learn the basics', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\nLearn the basics\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\n            \n            \n  To learn more about LangGraph, check out our first LangChain Academy course, Introduction to LangGraph, available for free here.\n\n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Learn the basics\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  Home\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHu

In [7]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=300)
splits= text_splitter.split_documents(doc)

In [8]:
len(splits)

123

In [9]:
vectorstore= FAISS.from_documents(documents=splits,embedding=embedding_model)

In [10]:
retriever = vectorstore.as_retriever()

In [11]:
retriever.invoke("what is langgraph?")

[Document(id='9548bda7-5c1a-4aa9-abce-20082d77795f', metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/introduction/#part-3-adding-memory-to-the-chatbot', 'title': 'Learn the basics', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content="4. LangGraph Studio: This appears to be a new tool in the LangGraph ecosystem, likely providing a graphical interface for working with LangGraph.\n\n5. Documentation: The LangGraph documentation has been revamped, which should make it easier for learners like yourself to understand and use the tool.\n\n6. Integration with LangSmith: While not directly part of LangGraph, LangSmith (another tool in the LangChain ecosystem) now features self-improving evaluators, which might be relevant if you're using LangGraph as part of a larger LangChain project.\n\nAs you're learning LangGraph, it would be beneficial to:"),
 Document(id='386d078a-1e87-4830-8ff7-45e6124cc6a0', metadata={'source': 'https://langchain-ai

In [12]:
system_prompt= '''You are a smart AI assistant.Response based on Context.\n\nContext:\n\n{context}'''

In [13]:
chat_prompt= ChatPromptTemplate.from_messages([("system",system_prompt),
                                               ("human","{input}")])

In [14]:
stuff_chaining= create_stuff_documents_chain(llm=llm,prompt=chat_prompt)

In [15]:
rag_chain= create_retrieval_chain(retriever=retriever,combine_docs_chain=stuff_chaining)

In [16]:
response= rag_chain.invoke({"input":"What is Langgraph?"})

In [17]:
print(response['answer'])

LangGraph is a tool in the LangChain ecosystem, which is a leading platform in AI development. It appears to be a comprehensive platform that provides various features and tools for working with AI, including a graphical interface, cloud-based version, server, command line interface, and software development kit.

LangGraph is designed to be more reliable, extensible, and advanced compared to simpler autonomous agent approaches. It offers a flexible architecture that allows users to easily add new features or modify existing ones, making it a powerful tool for building complex AI agents.

Some of the key features and components of LangGraph include:

1. LangGraph Cloud: A cloud-based version of LangGraph.
2. LangGraph Platform: A combination of several offerings, including LangGraph Server, LangGraph Studio, CLI, and SDK.
3. LangGraph Server: A component that has received new features to enhance its value proposition.
4. LangGraph Studio: A new tool that provides a graphical interface 

# History Aware Chat:

In [18]:
retriever_prompt= '''Retrieve contextually relevant historical information that accurately addresses the user’s query. 
Focus on key events, figures, and timelines, ensuring that sources are reliable and that the information reflects both historical facts and their 
contemporary significance.'''

In [19]:
prompt_with_history= ChatPromptTemplate.from_messages([("system",retriever_prompt),
                                                    MessagesPlaceholder(variable_name="chat_history"),
                                                    ("human","{input}"),])

In [20]:
history_aware_retriever= create_history_aware_retriever(llm=llm,retriever=retriever,prompt=prompt_with_history)

In [21]:
qa_prompt= ChatPromptTemplate.from_messages([("system",system_prompt),
                                             MessagesPlaceholder(variable_name="chat_history"),
                                               ("human","{input}")])

In [22]:
qa_chain= create_stuff_documents_chain(llm=llm,prompt=qa_prompt)

In [23]:
rag_chain= create_retrieval_chain(history_aware_retriever,qa_chain)

In [24]:
chat_history=[]

In [25]:
question1= "What is Langgraph?"

In [26]:
response1= rag_chain.invoke({"input":question1,"chat_history":chat_history})

In [27]:
print(response1['answer'])

LangGraph is a tool that is part of the LangChain ecosystem, a leading platform in AI development. It was officially released on January 17, 2024. LangGraph has recently been updated with new features and components, including:

1. LangGraph Cloud: A cloud-based version of LangGraph.
2. LangGraph Platform: A combination of several offerings - LangGraph Server, LangGraph Studio, CLI, and SDK.
3. LangGraph Server: This component has received new features to enhance its value proposition.
4. LangGraph Studio: A new tool in the LangGraph ecosystem, providing a graphical interface for working with LangGraph.

Additionally, LangGraph is considered more reliable and extensible than simpler autonomous agent approaches, offering advanced capabilities for building complex AI agents.

When getting started with LangGraph, you might want to explore the documentation, which has been revamped for better understanding and use. You can also utilize LangGraph Studio for a graphical interface and integra

In [28]:
chat_history.extend([HumanMessage(content=question1),
                     AIMessage(content=response1['answer'])])

In [29]:
question2= "No its coding based please write code."

In [30]:
response2= rag_chain.invoke({"input":question2,"chat_history":chat_history})

In [31]:
print(response2['answer'])

Sure, here's a basic example of how you might use LangGraph to define a simple chatbot that can answer questions and maintain conversation state:

```python
import langchain
from langchain.agents import Agent
from langchain.memory import SimpleMemory

# Initialize the LangGraph agent with a simple memory
agent = Agent(SimpleMemory())

# Define a function to handle user input
def handle_input(user_input):
    # Use the LangGraph agent to process the user input and generate a response
    response = agent.run(user_input)
    return response

# Example usage
user_input = "What is the release date of LangGraph?"
response = handle_input(user_input)
print(response)
```

This is just a simple example to get you started. LangGraph offers many more advanced features and capabilities for building complex AI agents.

You can also use LangGraph with other tools, such as LangSmith, to create more sophisticated AI applications. For example, you might use LangSmith to define self-improving evaluators

# Mehtod 2 for Chat History:

In [32]:
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [33]:
store={}

In [34]:
def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

In [35]:
model_with_memory= RunnableWithMessageHistory(llm,get_session_history=get_by_session_id)

In [36]:
config= {"configurable":{"session_id":"firstchat"}}

In [49]:
input_data = {
    "messages": [
        {"role": "human", "content": "What is my name"}
    ]
}

response = model_with_memory.invoke(input=input_data, config=config)
print(response.content)

Your name is Shorya, as you mentioned earlier. It's nice to see you again. Is there something on your mind that you would like to ask or discuss about artificial intelligence, machine learning, or data science? I'm here to help and provide information and insights on these topics. Don't hesitate to let me know if you have any questions or if there's something you would like to know more about. I'm here to assist you.


In [50]:
store['firstchat']

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! It's nice to User-15292 to see you here. How can I help you today? Is there something specific you would like to know or discuss about artificial intelligence, machine learning, or data science? I'm here to answer your questions and provide information and insights on these topics. Let me know how I can assist you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 8, 'total_tokens': 83, 'completion_time': 0.113842143, 'prompt_time': 0.001886005, 'queue_time': 0.037811512, 'total_time': 0.115728148}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-e26c5aa4-e110-4e16-aa53-b8a0c4248e4e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 75, 'total_tokens': 83}), HumanMessage(content='Hello', additional_kwargs={}, response

# RAG + ChatPromptTemplate + Multiple input variables:

In [52]:
llm= llm

In [294]:
system= '''Hi You are a smart chatbot.Always Answer based on given things.
            Topic:
            {topic}
            
            Method:
            {method}'''

In [295]:
prompt= ChatPromptTemplate.from_messages([("system",system),
                                                    MessagesPlaceholder(variable_name="chat_history"),
                                                    ("human","{input}")])

In [296]:
memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history",return_messages=True)

In [297]:
memory.buffer

[]

In [298]:
memory.load_memory_variables({})['chat_history']

[]

In [299]:
memory.load_memory_variables({})

{'chat_history': []}

In [300]:
topic=""
method=""
input= "Now how we make it 10."
chat_history=memory.load_memory_variables({})['chat_history']

In [301]:
final_prompt= prompt.format(topic=topic,method= method,input=input,chat_history=chat_history)

In [303]:
response=llm.invoke(final_prompt).content

In [304]:
print(response)

To make it 10, we need to know what "it" refers to. Could you please provide more context or information about what you want to make 10?


In [305]:
memory.save_context(inputs={HumanMessage:final_prompt},outputs={AIMessage:response})

In [306]:
memory.buffer

[HumanMessage(content='System: Hi You are a smart chatbot.Always Answer based on given things.\n            Topic:\n            \n            \n            Method:\n            \nHuman: Now how we make it 10.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='To make it 10, we need to know what "it" refers to. Could you please provide more context or information about what you want to make 10?', additional_kwargs={}, response_metadata={})]

# Memory+RAG+ChatPromptTemplate+Chain:

In [341]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 2})

# Define the system prompt template
system = '''Hi, you are a smart chatbot. Always answer based on the given things.
            Topic: {topic}
            Method: {method}
            {context}'''

prompt= ChatPromptTemplate.from_messages([("system",system),
                                                    MessagesPlaceholder(variable_name="chat_history"),
                                                    ("human","{input}")])

# Set up memory (to store conversation history)
memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)

# Create stuff documents chain
stuff_chaining = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Create the RAG (retrieval-augmented generation) chain
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=stuff_chaining)


In [342]:
# Prepare input data
topic = ""  # Set your topic
method = ""  # Set your method
input_text = "What is biology? answer based on context only"
chat_history = memory.load_memory_variables({})['chat_history']

# Retrieve context from the retriever
context = retriever.invoke(input_text)  # Assuming it retrieves the relevant documents

In [343]:
# Format the prompt with the contextual input (still using a dictionary format)
final_prompt = {
    "topic": topic,
    "method": method,
    "input": input_text,
    "chat_history": chat_history,
    "context": context  # Pass context as part of the dictionary
}

# Generate the final response using the RAG chain
response = rag_chain.invoke(final_prompt)

# Format the prompt with the contextual input
input_string = prompt.format(topic=topic, method=method, input=input_text, chat_history=chat_history, context=context)

memory.save_context(inputs={HumanMessage:input_string},outputs={AIMessage:response['answer']})

In [344]:
print(response['answer'])

There is no context about biology in the given text. The text appears to be a code snippet related to human assistance and verification of personal information, such as name and birthday.


# LCEL + Runnables + RAG + Chain:

In [523]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [524]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={'k': 2})

# Define the system prompt template
system = '''Hi, you are a smart chatbot. Always answer based on the given things.
            Topic: {topic}
            Method: {method}
            {context}'''

prompt= ChatPromptTemplate.from_messages([("system",system),
                                                    MessagesPlaceholder(variable_name="chat_history"),
                                                    ("human","{input}")])

# Set up memory (to store conversation history)
memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)

In [525]:
# Combine everything into a retriever chain
retriever_chain = RunnableParallel({
    "context": itemgetter("input") | retriever ,
    "topic": itemgetter("topic") ,  
    "method": itemgetter("method") ,  
    "input": itemgetter("input"),  
    "chat_history": itemgetter("chat_history")
}) | prompt | llm | StrOutputParser()

In [526]:
topic = "AI"  # Set your topic
method = "Code"  # Set your method
input_text = "What is Langgraph?"
chat_history = memory.load_memory_variables({})['chat_history']

In [527]:
# Now you can run the retriever_chain with the appropriate input
result = retriever_chain.invoke({
    'topic': topic,
    'method': method,
    'input': input_text,  # Use 'input' instead of 'input_text'
    'chat_history': chat_history
})

# Format the prompt with the contextual input
input_string = prompt.format(topic=topic, method=method, input=input_text, chat_history=chat_history, context=retriever.invoke(input_text))

memory.save_context(inputs={HumanMessage:input_string},outputs={AIMessage:result})


In [528]:
print(result)

LangGraph is part of the LangChain ecosystem, which is a leading platform in AI development. It allows users to build language agents as graphs. The LangGraph ecosystem includes several components, such as:

1. LangGraph Cloud: A cloud-based version of LangGraph.
2. LangGraph Platform: A combination of several offerings, including LangGraph Server, LangGraph Studio, CLI (Command Line Interface), and SDK (Software Development Kit).
3. LangGraph Server: A component that has received new features to enhance its value proposition.
4. LangGraph Studio: A new tool that provides a graphical interface for working with LangGraph.

LangGraph is used to build language agents as graphs, and it is part of a larger ecosystem that includes other tools like LangSmith, which features self-improving evaluators.
